In [ ]:
import os

import mindspore
from mindspore.dataset import text, GeneratorDataset, transforms
from mindspore import nn

from mindnlp import load_dataset

from mindnlp.engine import Trainer, Evaluator
from mindnlp.engine.callbacks import CheckpointCallback, BestModelCallback
from mindnlp.metrics import Accuracy

In [2]:
imdb_ds = load_dataset('imdb', split=['train', 'test'])
imdb_train = imdb_ds['train']
imdb_test = imdb_ds['test']

In [3]:
imdb_train.get_dataset_size()

25000

In [4]:
from mindnlp.transformers import AutoTokenizer
# tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

test_tokenized = tokenizer('hello')
test_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [5]:
import numpy as np

def process_dataset(dataset, tokenizer, max_seq_len=256, batch_size=32, shuffle=False):
    is_ascend = mindspore.get_context('device_target') == 'Ascend'
    def tokenize(text):
        if is_ascend:
            tokenized = tokenizer(text, padding='max_length', truncation=True, max_length=max_seq_len)
        else:
            tokenized = tokenizer(text, truncation=True, max_length=max_seq_len)
        return tokenized['input_ids'], tokenized['token_type_ids'], tokenized['attention_mask']

    if shuffle:
        dataset = dataset.shuffle(batch_size)

    # map dataset
    dataset = dataset.map(operations=[tokenize], input_columns="text", output_columns=['input_ids', 'token_type_ids', 'attention_mask'])
    dataset = dataset.map(operations=transforms.TypeCast(mindspore.int32), input_columns="label", output_columns="labels")
    # batch dataset
    if is_ascend:
        dataset = dataset.batch(batch_size)
    else:
        dataset = dataset.padded_batch(batch_size, pad_info={'input_ids': (None, tokenizer.pad_token_id),
                                                             'token_type_ids': (None, 0),
                                                             'attention_mask': (None, 0)})

    return dataset

In [6]:
# split train dataset into train and valid datasets
imdb_train, imdb_val = imdb_train.split([0.7, 0.3])

In [7]:
dataset_train = process_dataset(imdb_train, tokenizer, shuffle=True)
dataset_val = process_dataset(imdb_val, tokenizer)
dataset_test = process_dataset(imdb_test, tokenizer)

In [8]:
next(dataset_train.create_tuple_iterator())

[Tensor(shape=[32, 256], dtype=Int64, value=
 [[ 101, 2091, 1233 ... 1133, 1103,  102],
  [ 101, 1170, 3195 ...    0,    0,    0],
  [ 101,  146, 1148 ...    0,    0,    0],
  ...
  [ 101,  146, 1821 ...    0,    0,    0],
  [ 101,  146, 1486 ...    0,    0,    0],
  [ 101, 1258, 1515 ...    0,    0,    0]]),
 Tensor(shape=[32, 256], dtype=Int64, value=
 [[0, 0, 0 ... 0, 0, 0],
  [0, 0, 0 ... 0, 0, 0],
  [0, 0, 0 ... 0, 0, 0],
  ...
  [0, 0, 0 ... 0, 0, 0],
  [0, 0, 0 ... 0, 0, 0],
  [0, 0, 0 ... 0, 0, 0]]),
 Tensor(shape=[32, 256], dtype=Int64, value=
 [[1, 1, 1 ... 1, 1, 1],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  ...
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0]]),
 Tensor(shape=[32], dtype=Int32, value= [0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 
  1, 0, 0, 1, 1, 0, 1, 0])]

In [10]:
from mindnlp.transformers import AutoModelForSequenceClassification
from mindspore.experimental.optim import Adam

# set bert config and define parameters for training
model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased', num_labels=2)

optimizer = nn.Adam(model.trainable_params(), learning_rate=2e-5)

metric = Accuracy()

# define callbacks to save checkpoints
ckpoint_cb = CheckpointCallback(save_path='checkpoint', ckpt_name='bert_imdb_finetune', epochs=1, keep_checkpoint_max=2)
best_model_cb = BestModelCallback(save_path='checkpoint', ckpt_name='bert_imdb_finetune_best', auto_load=True)

trainer = Trainer(network=model, train_dataset=dataset_train,
                  eval_dataset=dataset_val, metrics=metric,
                  epochs=3, optimizer=optimizer, callbacks=[ckpoint_cb, best_model_cb],
                  jit=False)

The following parameters in checkpoint files are not loaded:
['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
The following parameters in models are missing parameter:
['classifier.weight', 'classifier.bias']


In [11]:
trainer.run(tgt_columns="labels")

The train will start from the checkpoint saved in 'checkpoint'.


Epoch 0: 100%|██████████████████████████████████████████████████████████████████████████████████| 547/547 [04:20<00:00,  2.10it/s, loss=0.32509542]


Checkpoint: 'bert_imdb_finetune_epoch_0.ckpt' has been saved in epoch: 0.


Evaluate: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 235/235 [00:40<00:00,  5.82it/s]


Evaluate Score: {'Accuracy': 0.9309333333333333}
---------------Best Model: 'bert_imdb_finetune_best.ckpt' has been saved in epoch: 0.---------------


...

Evaluate Score: {'Accuracy': 0.9690666666666666}
---------------Best Model: 'bert_imdb_finetune_best.ckpt' has been saved in epoch: 1.---------------


Epoch 2: 100%|███████████████████████████████████████████████████████████████████████████████████| 547/547 [04:19<00:00,  2.11it/s, loss=0.1257236]


The maximum number of stored checkpoints has been reached.
Checkpoint: 'bert_imdb_finetune_epoch_2.ckpt' has been saved in epoch: 2.


Evaluate: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 235/235 [00:40<00:00,  5.78it/s]


Evaluate Score: {'Accuracy': 0.9785333333333334}
---------------Best Model: 'bert_imdb_finetune_best.ckpt' has been saved in epoch: 2.---------------
Loading best model from 'checkpoint' with '['Accuracy']': [0.9785333333333334]...
---------------The model is already load the best model from 'bert_imdb_finetune_best.ckpt'.---------------


In [12]:
evaluator = Evaluator(network=model, eval_dataset=dataset_test, metrics=metric)
evaluator.run(tgt_columns="labels")

Evaluate: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 782/782 [02:15<00:00,  5.76it/s]

Evaluate Score: {'Accuracy': 0.91264}
